<a href="https://colab.research.google.com/github/elk-cloner/RL/blob/master/CartPole_Simple_DQN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np